In [5]:
import nltk
from nltk.tag import pos_tag
from nltk.corpus import twitter_samples
from nltk.stem.wordnet import WordNetLemmatizer
import re, string
from nltk import FreqDist
import random
import psycopg2
from textblob import TextBlob 
from nltk.tokenize import word_tokenize
import pandas as pd

In [6]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('twitter_samples')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\alexi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\alexi\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [7]:
text = twitter_samples.strings('tweets.20150430-223406.json')
twitter_samples

<TwitterCorpusReader in 'C:\\Users\\alexi\\AppData\\Roaming\\nltk_data\\corpora\\twitter_samples'>

In [8]:
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
print(pos_tag(tweet_tokens[0]))

[('#FollowFriday', 'JJ'), ('@France_Inte', 'NNP'), ('@PKuchly57', 'NNP'), ('@Milipol_Paris', 'NNP'), ('for', 'IN'), ('being', 'VBG'), ('top', 'JJ'), ('engaged', 'VBN'), ('members', 'NNS'), ('in', 'IN'), ('my', 'PRP$'), ('community', 'NN'), ('this', 'DT'), ('week', 'NN'), (':)', 'NN')]


In [9]:
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

print(lemmatize_sentence(tweet_tokens[0]))

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


In [10]:
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [7]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

print(remove_noise(tweet_tokens[0], stop_words))

['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']


In [8]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [9]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)


freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


In [10]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

In [11]:
positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

In [12]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

Accuracy is: 0.9956666666666667
Most Informative Features
                      :) = True           Positi : Negati =   1648.4 : 1.0
                     sad = True           Negati : Positi =     25.4 : 1.0
                follower = True           Positi : Negati =     22.4 : 1.0
                     bam = True           Positi : Negati =     21.2 : 1.0
                     x15 = True           Negati : Positi =     18.2 : 1.0
                 welcome = True           Positi : Negati =     14.8 : 1.0
               community = True           Positi : Negati =     14.5 : 1.0
                  arrive = True           Positi : Negati =     14.0 : 1.0
                    glad = True           Positi : Negati =     13.1 : 1.0
               goodnight = True           Positi : Negati =     13.1 : 1.0
None


In [ ]:
## Save Classifier Model
import pickle

f = open('my_classifier.pickle', 'wb')
pickle.dump(classifier, f)
f.close()

In [13]:
p="datasucks"
conn = psycopg2.connect(user = "alexis",
                                  password = f"{p}",
                                  host = "127.0.0.1",
                                  port = "3306",
                                  database = "postgres")

In [14]:
cursor = conn.cursor()
tweet_list = []
followers = []
cursor.execute("select * from tweets")
tweets = cursor.fetchall()
if not tweets:
    print("empty")
for row in tweets:
    for col in row:
        if type(col) is dict:
            tweet_list.append(col['text'])
            followers.append(col['user']['followers_count'])

In [15]:
remove_list = [r'@[A-Za-z0-9]+','https?://[A-Za-z0-9./]+','\n','RT :',r'_[A-Za-z0-9]+','  ']

def clean_tweets(remove_list,tweet_list):
    for item in remove_list:
        tweet_list = [re.sub(item,'',tweet) for tweet in tweet_list]
    return tweet_list

cleaned = clean_tweets(remove_list,tweet_list)

In [16]:
tweet = []
sentiment = []
for each in cleaned:
    custom_tweet = each
    tweet.append(custom_tweet)
    custom_tokens = remove_noise(word_tokenize(custom_tweet))
    sentiment.append(classifier.classify(dict([token, True] for token in custom_tokens)))
                                         
    print(custom_tweet, classifier.classify(dict([token, True] for token in custom_tokens)))

There are so many hilarious comments in You Tubechat on @ show tonight. Positive
 Democrat frontrunner Michael Bloomberg says he wants to make the whole country "behave" while dressed as Mary Poppins.Wel… Positive
The latest COACH!#election2020 #misleadingtweets Negative
THERE IS NO ONE TO VOTE FOR! Centrists demand Valium to get them through #Election2020 Positive
RT : Calling all voters!! Are you ready for November?👉🏿RT to spread the word. 🗳️ #Election2020 http… Positive
 Team Warren! #PresidentWarren #DemocraticPrimaries #Election2020  Positive
RT :#Election2020 Enjoy… Negative
 As a man and feminist I am flabbergasted at this Tweet from you. This says so much about your own bi…  Positive
 It's $3 Thursday! I'm asking for everyone to help that is able to donate $3 to my campaign for Congress. Join us and help put p… Positive
2020 Presidential Election News Update🗞:2020 Democratic contenders plead for cash after draining their accounts…  Negative
 #Bloomberg is buying the Dem nominat

 I’m a purple county voter—my county is 1 of the 10 that could decide 2020. While I will #VoteBlueNoM… Negative
South Bend Residents Have a Message for America: Don't Elect Pete Buttigieg#Election2020 via  Positive
#freehugs &amp; and a great show! Tune in Sunday morning at 7 am, 104.1 KSGF #radiochicks #sarahandtiffany…  Positive
 #RussianCollusion Time to act now. After #election2020 may be too late! … Positive
 Ok Christians!We are not ashamed of the gospel and boldly proclaim our Lord and Savior Jesus.Let’s flip this poll.Ple… Positive
 WOW.. 😱😱🚨 👉American flag is REMOVED before debate begins. 💥This is a slap in the face to every soldier who died defending thi… Negative
The Lancet;Health Policy Volume 395, ISSUE 10223, P525-533, February 15, 2020 DOI:3/2…  Positive
 What I predict #Election2020  Negative
The best science for bettering lives. Explore the latest high-quality research from The Lancet family of journals.…  Positive
It is among the world's oldest, most prestigious, and 

Rep. Keith Ellison Demands Examples of 'Bernie Bro' Violence. Then Steve Scalise Stepped to the Mic.…  Positive
 Bloomberg dismisses past comments as ‘a joke.’ NDA shows plaintiff barred from offering differing view. #Election2020 http… Positive
 Bloomberg dismisses past comments as ‘a joke.’ NDA shows plaintiff barred from offering differing view. #Election2020 http… Positive
Strong pitch: I'll buy the Presidency. You folks spend your money down-ballot.Flaw: Can't beat Trump.…  Positive
Didn't take long to have them come to try to dispute this, but instead of providing information about who is on the…  Negative
 I'd vote for Kermit the Frog before these two guys or ! #FactsMatter #DemDebate #NevadaCaucus #Decision2020 #… Positive
Personally,I prefer president's who aren't mentally ill.#RepublicanParty #Republicans #Senate #voters #Election2020 Negative
we calculate that a single-payer, universal health-care system is likely to lead to a 13% savings in national healt…  Positive
 No, bu

Raise your hand if writing something even resembling criticism of Bernie Sanders gets you more angry comments than…  Positive
✅ REVIEW: A MUST READ MYSTERYThe cash cravings of the First Family🔴 'DARK TOWERS: Deutsche Bank, Donald Trump, a…  Negative
 We need to save ourselves and stat. #Election2020 Negative
For the Nevada Caucuses and all upcoming primaries. Say “Alexa open Speak Up America” to receive a text with your p…  Negative
Duh. Positive
 The truth that #msnbc n #CNN won’t show you!#socialism #socialismsucks #berniesanders #bernie2020 #bernie #medicaid #medica… Negative
 No body can know that. No one thought trump could come even close to winning. All bets are off. Many "no…  Negative
 Get this  Negative
 How will you cast your VOTE in the 2020 Presidential election if your choice is NOT the candidate?#Bernie #Biden #Bloo… Positive
 Where have we heard Russia before.... 🤔JK. They're pivoting back. #Election2020  Positive
This could be the race for president of the United State

 Imagine that! Nothing to back up #Russia 2.0 claims! No evidence!Fancy, that!Who’s surprised?😲Mildly shocked?🙀Y… Positive
 Which #JasonJohnson? THIS Jason Johnson? The one Yvette Carnell eviscerated a while back? 😏 #ados #reparations #election2… Negative
 Trump is not a king, and voters will prove that to him in November. #Election2020 Positive
 Floridais requiring that all persons seeking employment in the State be confirmed e-verified legal resi… Positive
In 2016, I gave my vote to black women by voting as black activist groups recommended. Doing it again this year. Th…  Negative
RT : #Election2020 #Decision2020  Negative
Ever notice how rich people who support socialism are hypocrites, but poor people who support socialism are thieves…  Negative
 Our nonpartisan Election Protection team () makes sure nothing stands in the way of Americans’ freedom to vote.… Negative
#rickastley for #president #election2020 @ Las Vegas, Nevada  Positive
 "Elizabeth Warren is just one prominent ex-co

...

 Your getting scared and weak, had to give us Nancy Pelosi high school bully name muthafucker.You…  Negative
 The 2020 primary election is coming up. Make sure your voter registration information is correct and up-to-date before you… Negative
 Russia commits a brutal cyberattack against the country of Georgia. Putin's Puppet DJT responds b… Negative
 Tom Perez won’t be part of that discussion- some of us wanted it for this cycle and he said no. He’s the most disastrous DN… Positive
 Do you believe America would be better off if more Christians served in elected office?#Retweet #Poll #Election2020 #… Positive
#Election2020: Dumping #Trump, #ClintEastwood backs #Bloomberg!!!:-))) Positive
 BREAKING:Brand new polling in #MO2 showscontinues to crush the latestballot filler.No won… Positive
Just so you know, in case you're confused or have bought into all the bollocks recently:Sinn Fein have as much of…  Negative
 Don't take your vote with you!Check your vote and put your vote in the ballot

 "Socialists don’t like people to do things for themselves. Socialists like to get people dependent on the state. You never build a… Negative
 🔁#Kansas U.S. Senate #Election2020-Nov-3.Support BARBARA BOLLIER #KSSEN, &amp; 👉RT!#YesWe… Negative
WHO Won't be fooled again.Change comes from the bottom up.No Oligarchs.#AggressiveProgressive Because we d…  Negative
 Excited to have received the endorsement of , which represents more than 310,000 teachers in #CA.I can't wait t… Positive
 BLOG: Cyber issues to look out for in the run up to the US Presidential Election#CyberCrime #US… Positive
 Let’s flip this poll, patriots. More Christians, less ANTI American Muslims in our Congress 🙏🇺🇸❤️🐾 Negative
RT : Just voted! #Election2020 #Changes  Positive
#Election2020#DONews #USA ELECTION2020 Positive
 Floridais requiring that all persons seeking employment in the State be confirmed e-verified legal resi… Positive
 Attn #California!NPP ballots are being sent to registered Dems. DO NOT CAST THAT BALLO

 Ok Christians!We are not ashamed of the gospel and boldly proclaim our Lord and Savior Jesus.Let’s flip this poll.Ple… Positive
According to intelligence officials, Russia is attempting to meddle in #Election2020. Anyone else feeling déjà vu?…  Positive
 I'm even more committed to supporting good candidates for #election2020. Positive
 The results of this poll so far are NOT indicative of any community I’ve ever lived in, or even visited 🤔🙄 Let’s vote! Positive
Withfavored to claim another victory in the Nevada caucuses on Saturday, some Democrats are frantic…  Negative
 #WakeUpAmericaAre DemocRats Mathematically Challenged?Do they really think life will be better under Socialism?Dupe… Negative
 🇺🇸🇺🇸Record number of candidates filed to run for office in House &amp; SenateBreaking previous numbers from same period.… Negative
 Wait!Russian collusion, oh my. Wrong guy! This is not how it was scripted by the DNC! Gee, why would Russia want Bernie… Negative
 Attn #California!NPP ballots ar

 I am going to run for President. My initial idea is that all Americans —regardless of age— shall be entitled to “S… Positive
 Unbelievable. 😡 Negative
 Do you believe America would be better off if more Christians served in elected office?#Retweet #Poll #Election2020 #… Positive
 #WakeUpAmericaAre DemocRats Mathematically Challenged?Do they really think life will be better under Socialism?Dupe… Negative
 #WakeUpAmericaAre DemocRats Mathematically Challenged?Do they really think life will be better under Socialism?Dupe… Negative
 Attn #California!NPP ballots are being sent to registered Dems. DO NOT CAST THAT BALLOT. There are no presidential candid… Negative
 Unbelievable. 😡 Negative
Last time I checked Clint Eastwood didn’t have good judgement when it came to Presidents. Is he still talking to the chair???#Election2020 Negative
 NO- it was U.S. Taxpayers on Main Street who bailed out the Auto Industry and pulled the USA out of…  Negative
 Ok Christians!We are not ashamed of the gospe

 Attn #California!NPP ballots are being sent to registered Dems. DO NOT CAST THAT BALLOT. There are no presidential candid… Negative
Billboards in North Texas feat. Mike Bloomberg saying he’ll “end the chaos.” That’s not possible. It’s like fightin…  Negative
Let's go people.We can't let satan win anything Positive
 #WakeUpAmericaAre DemocRats Mathematically Challenged?Do they really think life will be better under Socialism?Dupe… Negative
 Make sure you vote and share :) Positive
 Ok Patriots! You know what to do. This needs to be flipped and hard. RT so we put a really big net out and show the true… Negative
 Which #JasonJohnson? THIS Jason Johnson? The one Yvette Carnell eviscerated a while back? 😏 #ados #reparations #election2… Negative
 Come on ! This is UNBELIEVABLE ! Conservative Christians please help flip this Poll ! 🙏🙏#CCOT Negative
 Do you believe America would be better off if more Christians served in elected office?#Retweet #Poll #Election2020 #… Positive
#PresidentWarren

In [17]:
clean_sentiment = pd.DataFrame(tweet, sentiment).reset_index()
clean_sentiment = clean_sentiment.rename(columns = {"index":"sentiment",0:"tweet"})
clean_sentiment.head()

,sentiment,tweet
0,Positive,There are so many hilarious comments in You Tu...
1,Positive,Democrat frontrunner Michael Bloomberg says h...
2,Negative,The latest COACH!#election2020 #misleadingtweets
3,Positive,THERE IS NO ONE TO VOTE FOR! Centrists demand ...
4,Positive,RT : Calling all voters!! Are you ready for No...


In [18]:
new = clean_sentiment.groupby("sentiment")["sentiment"].count()
new

sentiment
Negative    1133
Positive    1183
Name: sentiment, dtype: int64